In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

c:\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
"""하이퍼파라미터 설정하기.
"""

BATCH_SIZE = 64
LEARNING_RATE = 1e-3

In [3]:
"""훈련 및 테스트 데이터세트 준비하기.
"""

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [4]:
"""훈련 및 테스트 데이터로더 준비하기.
"""

train_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE)

In [5]:
"""신경망 만들기.
Linear > ReLU > Linear > ReLU > Linear
"""

class NeuralNetwork(nn.Module):

    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    """훈련 루프
    """
    size = len(dataloader.dataset)

    # 모델을 학습모드로 설정 (배치정규화batch normalization 및 드롭아웃dropout 레이어들에 중요함)
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        # X: 데이터, y: 정답 레이블

        # 추론 후 손실 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        loss.backward()
        
        # 역전파에서 수집된 변화도로 매개변수를 조정함.
        optimizer.step()

        # 변화도를 0으로 재설정
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss = loss.item()
            current = batch * BATCH_SIZE + len(X)
            print(f"iter: {batch:>5d}, loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [7]:
def test_loop(dataloader, model, loss_fn):
    """테스트 루프
    """
    
    # 모델을 평가모드로 설정한다 (배치정규화 및 드롭아웃 레이어들에 중요함)
    model.eval()

    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss = 0
    correct = 0

    # torch.no_grad 컨텍스트 매니저를 사용하여 변화도gradient를 계산하지 않도록 한다.
    # requires_grad=True로 설정된 텐서들의 불필요한 변화도 연산 및 메모리 사용량을 줄여준다.
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Accuracy: {(100*correct):>0.1f}% / Avg loss: {test_loss:>8f}")

In [8]:
"""훈련 및 평가
"""

# 장치 확인하기
device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

EPOCHS = 60
for t in range(EPOCHS):
    print(f"\nEpoch {t+1:>3d}\n---------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done")

Using cuda device

Epoch   1
---------
iter:     0, loss: 2.297871  [   64/60000]
iter:   100, loss: 2.291607  [ 6464/60000]
iter:   200, loss: 2.264092  [12864/60000]
iter:   300, loss: 2.263625  [19264/60000]
iter:   400, loss: 2.258106  [25664/60000]
iter:   500, loss: 2.204788  [32064/60000]
iter:   600, loss: 2.224358  [38464/60000]
iter:   700, loss: 2.175654  [44864/60000]
iter:   800, loss: 2.189482  [51264/60000]
iter:   900, loss: 2.149080  [57664/60000]
Accuracy: 41.2% / Avg loss: 2.147065

Epoch   2
---------
iter:     0, loss: 2.165552  [   64/60000]
iter:   100, loss: 2.153446  [ 6464/60000]
iter:   200, loss: 2.090282  [12864/60000]
iter:   300, loss: 2.105554  [19264/60000]
iter:   400, loss: 2.068941  [25664/60000]
iter:   500, loss: 1.994585  [32064/60000]
iter:   600, loss: 2.025207  [38464/60000]
iter:   700, loss: 1.936219  [44864/60000]
iter:   800, loss: 1.957595  [51264/60000]
iter:   900, loss: 1.872978  [57664/60000]
Accuracy: 55.9% / Avg loss: 1.874676

Epoch